# Übung 2:

## Aufgabe 2b: 

**Feature Engineering (Mittel: 15 Punkte)**
Verwendet in dieser Aufgabe das adult.csv Datenset. Entwickelt auf Basis dieses Datensets eine Strategie zur Vorverarbeitung der Daten und überlegt euch welche Features ihr benötigt, um ein gutes Klassifizierungsergebnis zu erreichen. Entwickelt drei eigene Features, die ihr entweder aus den bestehenden Attributen ableitet, oder über zusätzliche Daten (z.B. Eigenschaften des native-country) hinzufügt. Ziel eurer Strategie soll es sein, die Daten für Machine Learning vorzubereiten und einen passenden Estimator zu trainieren. Nehmt gegebenenfalls Optimierungen im Setup vor, um den Score eures Estimators zu verbessern.

benötigte Tools

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import networkx as nx
#from pipev import plot_pipeline

from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

Als erstes werden die Daten geladen und die einzelnen Features genau betrachtet:

**Data Loading & Exploration

In [2]:
dataset_location ="data_adult.csv"
dataset = pd.read_csv(dataset_location)
dataset.head(500)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
dataset.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
dataset.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


**Out- Input Features**

In [5]:
output_features = [
    "income"
]

input_features = [
    "age",
    "workclass",
    "fnlwgt",
    "education", 
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country"
]

train_X, test_X, train_y, test_y = train_test_split(
    dataset[input_features], 
    dataset[output_features],
    test_size=0.2
)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

((26048, 14), (6513, 14), (26048, 1), (6513, 1))

In [6]:
train_X

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
19087,31,Self-emp-not-inc,111423,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,55,United-States
11325,50,Self-emp-not-inc,312477,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,40,United-States
29953,17,Private,38611,11th,7,Never-married,Adm-clerical,Own-child,White,Female,0,0,23,United-States
21900,40,Private,147206,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States
25381,45,Private,178319,Bachelors,13,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
6899,26,Private,410240,11th,7,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States
25984,41,Local-gov,201435,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,Black,Male,0,0,40,United-States
31247,53,?,155233,12th,8,Married-civ-spouse,?,Wife,White,Female,0,0,40,Italy
16199,30,Private,114691,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States
13328,49,Self-emp-inc,141058,Some-college,10,Divorced,Exec-managerial,Not-in-family,White,Male,0,2339,50,United-States


In [7]:
from sklearn.pipeline import Pipeline

class FeaturePipeline(Pipeline):
    def get_feature_names(self):
        return self.steps[-1][1].get_feature_names()

In [8]:
class FeatureSelector:
    """This transformer lets you pick columns from a pandas dataset based on name"""
    def __init__(self, features = [], debug=False):
        self.d = debug
        if type(features) != list:
            raise ValueError("Input features must be of type List.")
        elif type(features) == list:
            self.columns = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X[self.columns]
        self.colnames = X.columns.tolist()
        return X
    
    def get_feature_names(self):
        return self.colnames

In [9]:
class adult_OneHotEncoder:
    def __init__(self, debug=False):
        self._encoder = None
        self._column_list = []
        self.d = debug
        self.colnames = None
        
    def fit(self, X, y=None):
        self._encoder = OneHotEncoder(sparse=False)
        self._encoder.fit(X)
        return self
    
    def transform(self, X):
        self._column_list=[]
        result_X= self._encoder.transform(X)
        for column in self._encoder.categories_:
            for value in column:
                self._column_list.append(value)
        X = pd.DataFrame(result_X, columns=self._column_list)
        self.colnames = X.columns.tolist()
        return X
    
    def get_feature_names(self):
        return self.colnames

In [10]:
class ContinentMapper:
    def __init__(self, debug=False):
        self.d = debug
        self.colnames = None
    
    def fit(self, X, y=None):
        return self 

    def transform(self, X):
        df = X.copy()
        country_dictionary = {
            "United-States":"North-America", 
            "Cambodia":"Asia", 
            "England":"Europe", 
            "Puerto-Rico":"South-America", 
            "Canada":"North-America", 
            "Germany":"Europe", 
            "Outlying-US(Guam-USVI-etc)":"North-America", 
            "India":"Asia",
            "Japan":"Asia", 
            "Greece":"Europe", 
            "South":"North-America", 
            "China":"Asia", 
            "Cuba":"North-America", 
            "Iran":"Asia", 
            "Honduras":"South-America", 
            "Philippines":"Asia", 
            "Italy":"Europe", 
            "Poland":"Europe", 
            "Jamaica":"South-America", 
            "Vietnam":"Asia", 
            "Mexico":"North-America", 
            "Portugal":"Europe", 
            "Ireland":"Europe", 
            "France":"Europe", 
            "Dominican-Republic":"South-America", 
            "Laos":"Asia", 
            "Ecuador":"South-America", 
            "Taiwan":"Asia", 
            "Haiti":"South-America", 
            "Columbia":"South-America", 
            "Hungary":"Europe", 
            "Guatemala":"South-America", 
            "Nicaragua":"South-America", 
            "Scotland":"Europe", 
            "Thailand":"Asia", 
            "Yugoslavia":"Europe", 
            "El-Salvador":"South-America", 
            "Trinadad&Tobago":"South-America", 
            "Peru":"South-America", 
            "Hong":"Asia", 
            "Holand-Netherlands":"Europe",
            "?":"unknown"
        }
        df["native-country"] = df["native-country"].map(country_dictionary).fillna("unknown")
        self.colnames = df.columns.tolist()
        return df
    
    def get_feature_names(self):
        return self.colnames

In [11]:
class AgeBinner:
    """Splits age into age groups and labels them"""    
    def __init__(self, debug=False):
        self.d = debug
        self.colnames = None
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df.age = pd.cut(
            df.age,
            bins=[0,16,30,45,150], 
            labels=["child", "young adult", "adult", "senior adult"]
        )
        self.colnames = df.columns.tolist()
        return df
    
    def get_feature_names(self):
        return self.colnames

In [12]:
class hours_per_week_Binner:
    """Splits age into age groups and labels them"""    
    def __init__(self, debug=False):
        self.d = debug
        self.colnames = None
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df["hours-per-week"] = pd.cut(
            df["hours-per-week"],
            bins=[0,1,39,41,150], 
            labels=["no work", "part-time", "full-time", "workaholic"]
        )
        self.colnames = df.columns.tolist()
        return df
    
    def get_feature_names(self):
        return self.colnames

In [13]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

debug_sate = False

pipe = Pipeline(
    [
        #("one_hot_encoder",OneHotEncoder(debug=False)),
        ("continent_mapper",ContinentMapper(debug=False)),
        ("age_binner", AgeBinner(debug=False)),
        ("hours-per-week", hours_per_week_Binner(debug=False)),
        ("feature_union",
            FeatureUnion(
            [
                ("non_num_pipe", FeaturePipeline(
                    [
                        ("non-numeric_selector", FeatureSelector(["age",
                                                                 "workclass",
                                                                 "education",
                                                                 "marital-status",
                                                                 "occupation", 
                                                                 "relationship",
                                                                 "race", 
                                                                 "sex", 
                                                                 "hours-per-week", 
                                                                 "native-country"], debug=False)),
                        ("one_hot_encoder", adult_OneHotEncoder()),
                    ]
                )),
                ("bypass_pipe", FeaturePipeline(
                        [
                            ("bypass_selector", FeatureSelector(["fnlwgt", "capital-gain", "capital-loss"], debug=False)),
                        ]
                    ))
            ]
            )
        )
    ]
)

In [14]:
pipe.fit(train_X)

Pipeline(memory=None,
     steps=[('continent_mapper', <__main__.ContinentMapper object at 0x125633be0>), ('age_binner', <__main__.AgeBinner object at 0x125633ba8>), ('hours-per-week', <__main__.hours_per_week_Binner object at 0x125633b70>), ('feature_union', FeatureUnion(n_jobs=None,
       transformer_list=[('non_num_pipe',...selector', <__main__.FeatureSelector object at 0x125633cc0>)]))],
       transformer_weights=None))])

In [15]:
train_X_prepared = pipe.transform(train_X)
test_X_prepared = pipe.transform(test_X)

In [16]:
train_X_prepared

array([[1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.11423e+05,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 3.12477e+05,
        0.00000e+00, 1.88700e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 3.86110e+04,
        0.00000e+00, 0.00000e+00],
       ...,
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.81863e+05,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.72186e+05,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 2.38401e+05,
        0.00000e+00, 0.00000e+00]])